In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import geopandas as gpd
import requests
import os

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data Cleaning")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/11 09:17:17 WARN Utils: Your hostname, DESKTOP-702MS12 resolves to a loopback address: 127.0.1.1; using 172.17.210.38 instead (on interface eth0)
24/09/11 09:17:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 09:17:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Join merchant & consumer data

In [3]:
def join_func(left: DataFrame, right: DataFrame, left_keys: ArrayType, 
              method: StringType='inner', right_keys: ArrayType=None, drop_left: BooleanType=False) -> DataFrame:
    """
    Joins two pyspark dataframes on the left and right specified keys, returning the inner join by default.
    Identifies records not joined in left and right datasets.
    """
    
    if not right_keys:
        right_keys = left_keys

    join_cond = "&".join([f"(left.{i} == right.{j})" for i,j in zip(left_keys, right_keys)])

    if not drop_left:
        drop_cols = [right[c] for c in right_keys]
    else: 
        drop_cols = [left[c] for c in left_keys]

    # Join
    joined_df = left.join(right, eval(join_cond), how=method).drop(*drop_cols)

    # See which return all the rows from left df that do not have a match in the right df
    mismatch = left.join(right, eval(join_cond), how='left_anti').drop(*drop_cols)
    print(f"Number of unjoined left records: {mismatch.count()}")

    return joined_df

In [4]:
# Load in merchant data (parquet)
merchant = spark.read.parquet("../data/tables/part_1/tbl_merchants.parquet")

# Load in merchant fraud (csv)
merchant_fp = pd.read_csv("../data/tables/part_1/merchant_fraud_probability.csv")
merchant_fp = spark.createDataFrame(merchant_fp)

# Load in consumer list (csv)
consumer = pd.read_csv("../data/tables/part_1/tbl_consumer.csv", delimiter="|")
consumer = spark.createDataFrame(consumer)

# Load in consumer fraud (csv)
consumer_fp = pd.read_csv("../data/tables/part_1/consumer_fraud_probability.csv")
consumer_fp = spark.createDataFrame(consumer_fp)

consumer_ud = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")

**Join merchants data**

`tbl_merchant` to `merchant_fraud_probability`

In [5]:
merchant_full = join_func(merchant, merchant_fp, ['merchant_abn'], 'left', drop_left=False)
merchant_full.orderBy('fraud_probability', ascending=True).show(5)

Number of unjoined left records: 3978
+--------------------+--------------------+------------+--------------+-----------------+
|                name|                tags|merchant_abn|order_datetime|fraud_probability|
+--------------------+--------------------+------------+--------------+-----------------+
|        Mollis Corp.|((watch, clock, a...| 12516851436|          NULL|             NULL|
|     Ante Industries|[[motor vehicle s...| 15613631617|          NULL|             NULL|
|Pellentesque Habi...|([cable, Satellit...| 19839532017|          NULL|             NULL|
|Mauris Nulla Inte...|((opticians, opti...| 34440496342|          NULL|             NULL|
|Quis Tristique Ac...|[(watch, clock, a...| 35344855546|          NULL|             NULL|
+--------------------+--------------------+------------+--------------+-----------------+
only showing top 5 rows



**Join customer data**

`tbl_consumer` to `consumer_user_detail` and `consumer_fraud_probability`

In [6]:
consumer_full = join_func(consumer, consumer_ud, ['consumer_id'], 'left') # match user_id to consumers
consumer_full = join_func(consumer_full, consumer_fp, ['user_id'], 'left') # match fraud probaibility to consumers

# Drop irrelevant columns
consumer_full = consumer_full.drop(*['address', 'order_datetime'])
consumer_full.show(10)

24/09/11 09:17:51 WARN TaskSetManager: Stage 18 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 0


24/09/11 09:17:54 WARN TaskSetManager: Stage 25 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 479871
+-----------------+-----+--------+------+-----------+-------+------------------+
|             name|state|postcode|gender|consumer_id|user_id| fraud_probability|
+-----------------+-----+--------+------+-----------+-------+------------------+
| Yolanda Williams|   WA|    6935|Female|    1195503|      1|  9.80543113652096|
|       Mary Smith|  NSW|    2782|Female|     179208|      2| 9.599513915425788|
|       Mary Smith|  NSW|    2782|Female|     179208|      2|10.069850934775245|
|    Jill Jones MD|   NT|     862|Female|    1194530|      3| 8.300636455314633|
|  Lindsay Jimenez|  NSW|    2780|Female|     154128|      4|  9.63330241109042|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|  9.02022421158597|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|10.868364868449886|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|27.496186536467164|
|    Karen Chapman|  NSW|    2033|Female|     407340|      6|10.45928

24/09/11 09:17:56 WARN TaskSetManager: Stage 32 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


**Join customers and transaction data**

In [7]:
# Read transaction dataset
transaction1 = spark.read.parquet("../data/tables/part_2")
transaction2 = spark.read.parquet("../data/tables/part_3")
transaction3 = spark.read.parquet("../data/tables/part_4")

transaction = transaction1.union(transaction2).union(transaction3)

In [8]:
# Join customers to transactions
consumer_transaction = consumer_full.join(transaction, on='user_id')

# Count number of customers in dataset with no transactions
mismatch_c_t = consumer_full.join(transaction, on='user_id', how='left_anti')
print(f"There are {mismatch_c_t.count()} customers with no matching transactions")

24/09/11 09:18:02 WARN TaskSetManager: Stage 42 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


There are 475918 customers with no matching transactions


In [9]:
consumer_full.select(['postcode', 'gender', 'user_id', 'fraud_probability']).join(transaction.select(['user_id', 'merchant_abn']), on='user_id')

24/09/11 09:18:09 WARN TaskSetManager: Stage 57 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.
24/09/11 09:18:13 WARN TaskSetManager: Stage 66 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


user_id,postcode,gender,fraud_probability,merchant_abn
7,4606,Female,NULL,33064796871
7,4606,Female,NULL,68435002949
7,4606,Female,NULL,41944909975
7,4606,Female,NULL,21439773999
7,4606,Female,NULL,86662713230
7,4606,Female,NULL,61447419161
7,4606,Female,NULL,60956456424
7,4606,Female,NULL,21439773999
7,4606,Female,NULL,80518954462
7,4606,Female,NULL,68559320474


In [10]:
consumer_transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



# Join external datasets

In [11]:
# Load in dataset
medians = pd.read_csv("../data/tables/sa2_dataset/main/C21_G02_SA2_filtered.csv")

# Column names
variables = {'REGION': 'sa2_code',
             1: "median_age", 
             2: "median_total_personal_income",
             3: "median_total_family_income",
             4: "median_total_household_income",
             5: "median_mortgage_repayment",
             6: "median_rent",
             7: "avg_people_per_bedroom",
             8: "avg_household_size"}

medians = medians.pivot(index='REGION', columns=['MEDAVG'], values='OBS_VALUE').reset_index().rename(columns=variables)
medians.columns.name = None
medians['sa2_code'] = medians.sa2_code.astype(str)

medians.head(5)

,sa2_code,median_age,median_total_personal_income,median_total_family_income,median_total_household_income,median_mortgage_repayment,median_rent,avg_people_per_bedroom,avg_household_size
0,101021007,51.0,760.0,1886.0,1429.0,1732.0,330.0,0.8,2.2
1,101021008,38.0,975.0,2334.0,1989.0,1950.0,350.0,0.8,2.6
2,101021009,37.0,996.0,2233.0,1703.0,1700.0,330.0,0.9,2.1
3,101021010,36.0,1104.0,2412.0,1796.0,1700.0,310.0,0.9,2.1
4,101021012,37.0,1357.0,3332.0,3014.0,2300.0,430.0,0.8,2.9


In [12]:
# Read in zone file to identify SA2 zones with null median/average values
sa2_boundary_gdf = gpd.read_file("../data/tables/sa2_boundary/SA2_2021_AUST_GDA2020.shp")
sa2_names = sa2_boundary_gdf[['SA2_CODE21', 'SA2_NAME21']].astype('string')

# Find records with null statistics
null_regions = medians[medians.isna().any(axis=1)]
null_regions = null_regions.merge(sa2_names, left_on='sa2_code', right_on='SA2_CODE21')
null_regions.iloc[:,-2:]

,SA2_CODE21,SA2_NAME21
0,199999499,No usual address (NSW)
1,299999499,No usual address (Vic.)
2,399999499,No usual address (Qld)
3,499999499,No usual address (SA)
4,599999499,No usual address (WA)
5,699999499,No usual address (Tas.)
6,799999499,No usual address (NT)
7,899999499,No usual address (ACT)
8,999999499,No usual address (OT)


24/09/10 20:54:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2674691 ms exceeds timeout 120000 ms
24/09/10 20:54:32 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/10 20:54:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [12]:
# Download data
url = """https://github.com/matthewproctor/australianpostcodes/blob/92a036281ee4009be03cca3ab0b8b1a49b21dca7/australian_postcodes.csv"""
base_path = "../data/tables/poa_dataset"

response = requests.get(url, headers={'accept': 'text/csv'}, stream=True)
response.raise_for_status()

# Create directory if it doesn't exist
if not os.path.exists(base_path):
    os.makedirs(base_path)

with open("../data/tables/poa_dataset/postcodes_to_sa2.csv", 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

In [ ]:
# get sa2 - postcodes
# write code to come up with useful statistics per 